In [19]:
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from newspaper import Article

In [24]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [25]:
links = []
def get_links(no_of_pages : int) -> list[str]:
    '''scrape article links from machine learning mastery blog'''
    for i in tqdm(range(no_of_pages)):
        url = f'https://machinelearningmastery.com/blog/page/{i+1}'
        r_page = requests.get(url, headers=headers)
        soup = BeautifulSoup(r_page.content, 'lxml')
        articles = soup.find_all('article')
        links.extend([article.find('a')['href'] for article in articles if article.find('a')])
    return links

In [26]:
articles_links = get_links(155)

  0%|          | 0/155 [00:00<?, ?it/s]

In [29]:
def get_article_data(article_links: list[str]) -> tuple[list[str], list[str], list[str], list[str], list[str]]:
    '''extract data from the article'''
    text, keywords, summary, title, published_date = [], [], [], [], []
    
    for link in tqdm(article_links):
        article = Article(link, language='en')
        article.download()
        article.parse()
        article.nlp()

        text.append(article.text)
        summary.append(article.summary)
        keywords.append(article.keywords)
        title.append(article.title)
        published_date.append(article.publish_date)

    return text, summary, keywords, title, published_date




In [30]:

text, summary, keywords, title, published_date = get_article_data(
    articles_links)

  0%|          | 0/1141 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
articles = pd.DataFrame({
    'text' : text,
    'summary' : summary,
    'keywords' : keywords,
    'title' : title,
    'published date': published_date,
    'article link' : articles_links
})
articles.to_csv('machine-mastery-articles.csv', index=False, encoding='utf-8')